In [1]:
import pandas as pd
import cx_Oracle

In [12]:
ip = '172.16.80.74'
port = '1521'
SID = 'RPTDB'

dsn_tns = cx_Oracle.makedsn(ip, port, SID)
connection = cx_Oracle.connect('ANLYUSR', 'ANLYUSR$', dsn_tns)

In [54]:
query = "SELECT tb1.STATE,tb1.PINCODE,tb1.PORTFOLIO,tb1.STATUS, COUNT(1) as STATUS_COUNT \
FROM TBL_TRN_CUSTOMER_DETAILS tb1 \
WHERE tb1.PORTFOLIO in ('TW','CD') \
AND tb1.STATUS in ('M','L') \
AND tb1.STATE in ('TN','AP','KA','MH','WB') \
AND tb1.PINCODE in (SELECT * FROM CODES_NEW) \
GROUP BY tb1.STATE,tb1.PINCODE,tb1.PORTFOLIO,tb1.STATUS"

In [55]:
Data = pd.read_sql(query, con=connection, chunksize=1000)

In [56]:
data = pd.concat([chunk for chunk in Data])

In [57]:
data.head()

,STATE,PINCODE,PORTFOLIO,STATUS,STATUS_COUNT
0,MH,413514,TW,L,53
1,TN,628552,TW,L,214
2,TN,628908,TW,L,34
3,TN,629176,TW,L,103
4,TN,600068,TW,M,197


In [58]:
totalCount = data.groupby(['PINCODE','PORTFOLIO']).agg({'STATUS_COUNT':'sum'}).reset_index()

In [59]:
totalCount.head()

,PINCODE,PORTFOLIO,STATUS_COUNT
0,400015,TW,9
1,400024,TW,9
2,400027,TW,5
3,400029,TW,1
4,400042,CD,1


In [60]:
data_v1 = pd.merge(data,totalCount, on=['PINCODE','PORTFOLIO'], how='left')

In [61]:
data_v1.rename(columns={'STATUS_COUNT_y':'TOTAL_COUNT'}, inplace=True)

In [62]:
data_v1 

,STATE,PINCODE,PORTFOLIO,STATUS,STATUS_COUNT_x,TOTAL_COUNT
0,MH,413514,TW,L,53,76
1,TN,628552,TW,L,214,529
2,TN,628908,TW,L,34,85
3,TN,629176,TW,L,103,281
4,TN,600068,TW,M,197,554
5,TN,604202,TW,L,216,382
6,TN,638501,TW,M,301,602
7,TN,636902,TW,L,141,283
8,TN,627756,TW,M,709,1249
9,AP,515004,TW,M,390,796


In [63]:
data_v1.to_csv("D:\PERCENTAGE_ML_CD_TW_Newset.csv", index=False)